In [1]:
# Set your API key here:

# import os
# os.environ['OPENAI_API_KEY'] = ''

## LLMChain

In [2]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

prompt_template = "Write a programming joke, make the joke about the following language: {language}?"

llm = ChatOpenAI()
llm_chain = LLMChain(
    llm=llm,
    prompt= ChatPromptTemplate.from_template(prompt_template)
)
llm_chain("Php")

{'language': 'Php',
 'text': "Why did the PHP developer go broke?\n\nBecause he couldn't stop echoing his money!"}

## LCEL

In [3]:
chain = ChatPromptTemplate.from_template(prompt_template) | ChatOpenAI(temperature=0)
chain.invoke({
    "language": "JavaScript"
})

AIMessage(content='Why did the JavaScript developer go broke?\n\nBecause he lost all his prototypes!')

------

## Sequential Chains for LLMChain

### LLMChain

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

In [5]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.
Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(input_variables=["title", "era"], template=synopsis_template)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt_template, output_key="synopsis")

In [6]:
# This is an LLMChain to write a review of a play given a synopsis.
review_template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
review_prompt_template = PromptTemplate(input_variables=["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=review_prompt_template, output_key="review")

In [7]:
llm = ChatOpenAI(temperature=.7)
social_template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, 
the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
social_prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=social_template)
social_chain = LLMChain(llm=llm, prompt=social_prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': '🌅 Don\'t miss the mesmerizing tale of forbidden love and heart-wrenching tragedy in "Tragedy at Sunset on the Beach"! 🎭✨ Step back in time to Victorian England and immerse yourself in a world of opulence, societal constraints, and the power of love. 💔❤️\n\nJoin us on December 25th at 8pm PST at the Theater in the Park as we transport you to a bygone era, where crashing waves and golden-hued skies set the stage for an unforgettable tragedy. 🌊🌅✨\n\nWitness the captivating performances of Amelia and William, two star-crossed lovers who find solace in their clandestine meetings on a secluded beach at sunset. Their love blossoms against the backdrop of societal judgment and rigid class divisions, but will they succumb to the suffocating grip of propriety or defy the odds? 🌹💔\n\nAccording to the New York Times, "Tragedy at Sunse

### LCEL

In [8]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

In [9]:
snyopsis_chain = (
    synopsis_prompt_template | ChatOpenAI() |  StrOutputParser() | { "synopsis": RunnablePassthrough(), "title": itemgetter("title"), "era": itemgetter("era") } 
)

review_chain = ({
        "review":  review_prompt_template | ChatOpenAI() | StrOutputParser() | RunnablePassthrough(),
        "synopsis": itemgetter("synopsis"),
        "title": itemgetter("title"),
        "era": itemgetter("era")
})


social_chain = (
    RunnableParallel(
        synopsis=itemgetter("synopsis"),
        review=itemgetter("review"),
        time=lambda x: "December 25th, 8pm PST",
        location=lambda x: 'London'
    ) | 
    social_prompt_template | ChatOpenAI() | StrOutputParser() | RunnablePassthrough()
)

sequential_chain = (
    synopsis_chain | review_chain | { "social_media_post": social_chain, 
                                      "synopsis": itemgetter("synopsis"), 
                                      "review": itemgetter("review"),
                                      "title": itemgetter("title"),
                                      "era": itemgetter("era")}
                    )

In [10]:
lcel_chain = sequential_chain.invoke({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})

In [13]:
print(lcel_chain.keys())

dict_keys(['social_media_post', 'synopsis', 'review', 'title', 'era'])


In [14]:
print(lcel_chain)

{'social_media_post': '🌅 Don\'t miss "Tragedy at Sunset on the Beach" - a captivating Victorian-era masterpiece that will transport you to the opulent shores of a seaside town. 🌊✨\n\nJoin us on December 25th at 8pm PST in London for a night of love, betrayal, and self-discovery. 🎭 Witness Lady Arabella\'s enthralling journey as she navigates societal expectations and her own desires, while a mysterious survivor, Edward, brings an unexpected twist to their lives. 🌹\n\nCritics are raving about this gripping tale that explores themes of sacrifice, the consequences of societal norms, and the true meaning of love. 📖 The New York Times praised the play\'s lush Victorian setting, compelling characters, and masterful storytelling. 🌟\n\nGet ready for an evening of heart-wrenching emotions, evocative scenes, and a cast of unique characters whose motives will keep you on the edge of your seat. 🎟️🔥\n\nReflect on the boundaries imposed by society and the price of happiness in a world driven by appe